In [1]:
import pandas as pd
import datetime
from ipywidgets import *
pd.set_option('display.max_colwidth', 128)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 60)

In [2]:
"""
-- Modified original query from https://dune.com/queries/92408/184718

SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID,
  tx.value/1e18 AS price_eth,
  date_trunc('second', mints."evt_block_time") AS time,   
  mints."_to" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721_evt_Mint" mints -- old contrct
LEFT JOIN ethereum.transactions tx
  ON mints."evt_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"

UNION ALL 
    
SELECT 
  tx.hash,
  tx.success,
  --pid."name", 
  mints."_projectId" AS ProjectID, 
  tx.value/1e18 AS price, 
  date_trunc('second', mints."call_block_time") AS time, 
  mints."_by" AS buyer, 
  (tx."gas_used" * tx."gas_price"/1e18) AS gas_eth
FROM artblocks."GenArt721Core_call_mint" mints -- new contract
LEFT JOIN ethereum.transactions tx
  ON mints."call_tx_hash" = tx."hash"
--LEFT JOIN dune_user_generated.ArtBlocksProjectIDs pid 
--  ON pid.id = mints."_projectId"
WHERE "output__tokenId" is not null
ORDER BY time DESC
"""

d = pd.read_csv('../mint.csv')
d["time"] = pd.to_datetime(d["time"])
display(d.dtypes)

# sort by time and descending gas for most probable execution order without looking at transaction order numbers.
d.sort_values(by=["time", "gas_eth"], ascending=[True, False], inplace=True)

d.head()

hash                      object
success                     bool
projectid                  int64
price_eth                float64
time         datetime64[ns, UTC]
buyer                     object
gas_eth                  float64
dtype: object

,hash,success,projectid,price_eth,time,buyer,gas_eth
206053,\xc86f9caf0307f66d63c03aa1952f47e1a7f09243a8e7e3b26faa374a60c253dc,True,2,0.10,2020-11-27 15:58:01+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.015308
206052,\x2b4c7709bcb24f5f0337fcf5c045a5cbc911ab118f8bd0439db52615aa12d2ad,True,2,0.10,2020-11-27 16:00:31+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.013010
206051,\x99666b7a136f58b78abbb8226bae746e51293c927d2efc8d3d2e4bb4f3c1f500,True,2,0.10,2020-11-27 16:08:37+00:00,\x7d42611012fdbe366bf4a0481fc0e1abf15e245a,0.015034
206050,\x148da1d93e382e3220df987557f0240f14e24772feb4435a7a0337d607da6cdf,True,1,0.05,2020-11-27 16:10:41+00:00,\xc7391970d642faf65fabac8f63b0d41c4481d787,0.017108
206049,\x9e812dec2467b3f9c84fc3e589cc608360284a51def0bcae675fd43bae4da26f,True,2,0.10,2020-11-27 16:11:28+00:00,\xc7391970d642faf65fabac8f63b0d41c4481d787,0.014986


In [3]:
display(d.info())
display("Number of successful mints: ", d.success.sum())
d.describe(include=['bool','float', 'int', 'datetime'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206054 entries, 206053 to 0
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   hash       206054 non-null  object             
 1   success    206054 non-null  bool               
 2   projectid  206054 non-null  int64              
 3   price_eth  206054 non-null  float64            
 4   time       206054 non-null  datetime64[ns, UTC]
 5   buyer      206054 non-null  object             
 6   gas_eth    206054 non-null  float64            
dtypes: bool(1), datetime64[ns, UTC](1), float64(2), int64(1), object(2)
memory usage: 11.2+ MB


None

'Number of successful mints: '

200028

,success,projectid,price_eth,gas_eth
count,206054,206054.000000,206054.000000,206054.000000
unique,2,NaN,NaN,NaN
top,True,NaN,NaN,NaN
freq,200028,NaN,NaN,NaN
mean,NaN,154.607181,0.458353,0.073694
std,NaN,99.534838,0.938460,0.142205
min,NaN,0.000000,0.000000,0.000000
25%,NaN,75.000000,0.100000,0.016263
50%,NaN,144.000000,0.120000,0.037782
75%,NaN,231.000000,0.294868,0.074062


In [4]:
types = pd.read_csv('../typebyname.csv')
display(types.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   label         146 non-null    object 
 1   eth_total     148 non-null    float64
 2   usd_total     147 non-null    float64
 3   eth_original  148 non-null    float64
 4   refund_eth    148 non-null    float64
 5   usd_original  148 non-null    float64
 6   refund_usd    148 non-null    float64
 7   mint_count    148 non-null    int64  
 8   projectid     148 non-null    int64  
 9   project_type  148 non-null    object 
dtypes: float64(6), int64(2), object(2)
memory usage: 11.7+ KB


None

In [5]:
mints = d[d["success"]]

import statistics

def getMiddleValue(pdSeries):
    mid = (pdSeries.count() / 2).astype(int)
    return pdSeries.iloc[mid]

types = types.set_index("projectid")
types = types["project_type"]
types = types.reset_index()
# adding project type to mint dataset
mints = pd.merge(mints, types, on="projectid", how="outer")
mintsByProjectId = mints.groupby("projectid")
mintsByProjectId = pd.DataFrame({
    "count": mintsByProjectId["projectid"].count(),
    "firstMintTime": mintsByProjectId["time"].first(),
    "lastMintTime": mintsByProjectId["time"].last(),
    "lastMintPriceTotal": mintsByProjectId["price_eth"].last() + mintsByProjectId["gas_eth"].last(),
    "minMintPrice": mintsByProjectId["price_eth"].min(),
	"medianMintPrice": mintsByProjectId["price_eth"].median(),
	"meanMintPrice": mintsByProjectId["price_eth"].mean(),
    "medianMintTime":  mintsByProjectId["time"].apply(lambda x: getMiddleValue(x)),
    "projectType": mintsByProjectId["project_type"].first()
    })
mintsByProjectId["latterMintWindowInMins"] = round((mintsByProjectId["lastMintTime"] - mintsByProjectId["medianMintTime"]).dt.total_seconds() / 60, 2)
mintsByProjectId["totalMintWindowInMins"] = ((mintsByProjectId["lastMintTime"] - mintsByProjectId["firstMintTime"]).dt.total_seconds() / 60)
mintsByProjectId["latterMintWindowUnder4Hours"] = (mintsByProjectId["latterMintWindowInMins"] <= 240)

In [6]:
mintsByProjectIdNoIndex = mintsByProjectId.reset_index()
mintsByProjectIdNoIndex = mintsByProjectIdNoIndex[mintsByProjectIdNoIndex["projectType"] == "Curated"]
settings = ["medianMintPrice", "meanMintPrice", "minMintPrice", "totalMintWindowInMins"]
def updateStats(i = 1):
	mintsByProjectIdNoIndex.plot.scatter(x="projectid", y=settings[i], figsize=(20,8), title="curated collections")
	mintsByProjectIdNoIndex[mintsByProjectIdNoIndex["latterMintWindowUnder4Hours"] == True].plot.scatter(x="projectid", y=settings[i], figsize=(20,8), title="latter mint window under 4 hours")

interact(updateStats)

interactive(children=(IntSlider(value=1, description='i', max=3, min=-1), Output()), _dom_classes=('widget-int…

<function __main__.updateStats(i=1)>

In [7]:
"""
-- Modified original query from https://dune.com/queries/160701/314169

select distinct block_time, 
  ROUND("nft_token_id"::numeric / 1000000) as projectid,
  round(eth_amount, 2) as eth_price, 
  usd_price, 
  link, 
  platform, 
  left(seller::text, 7) as seller, 
  left(buyer::text, 7) as buyer 
from 
(
select 
  block_time, 
  platform, 
  usd_amount, 
     
  case 
     when ("original_currency" = 'ETH' OR "original_currency" = 'WETH')
             THEN  ("original_amount")
    else 0  
  END as eth_amount, 
  "usd_amount" as usd_price,

   
 CONCAT('<a href="https://opensea.io/assets/', CONCAT('0x', substring(a."nft_contract_address"::text from 3)), '/', a.nft_token_id,  '/?ref=0x8F903cFC0Af3C2EC0d872c57538AF5e071544a57','" target="_blank" >', 'View on OS', '</a>') as  link,
   
 seller, 
 buyer, 
 tx_hash,
 nft_token_id

from nft."trades" a
WHERE 
     "trade_type" = 'Single Item Trade'
     AND (a.nft_contract_address = '\xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270'
    OR  a.nft_contract_address = '\x059edd72cd353df5106d2b9cc5ab83a52287ac3a')
ORDER BY block_time DESC 
) gg
-- WHERE block_time > '{{Date}}'
order by block_time DESC
"""

p = pd.read_csv('../sales.csv')

display("Before filtering:", len(d))

# cleaning up weird project ids
#p_removed = p[p["projectid"].str.len() >= 8]
#p = p[p["projectid"].str.len() < 8]

# casting
p["time"] = pd.to_datetime(p["time"])
p["projectid"] = p["projectid"].astype(int)

p.sort_values(by=["time"], ascending=[True], inplace=True)

#removing non valid transactions
p = p[p["eth_total"] > 0]

# adding derived data
p["normalized_price"] = p["eth_total"] / p["projectid"].map(mintsByProjectId.lastMintPriceTotal)
p["latterMintWindowInMins"] = p["projectid"].map(mintsByProjectId.latterMintWindowInMins)
p["lastMintTime"] = p["projectid"].map(mintsByProjectId.lastMintTime)
p["isWithin2hFromLastMintTime"] = ((p["time"] - p["lastMintTime"]).dt.total_seconds() / 60 < 120) & ((p["time"] - p["lastMintTime"]).dt.total_seconds() > 0)

display(p.dtypes)
display(p.describe(include=['bool','float', 'int', 'datetime']))
p

'Before filtering:'

206054

time                          datetime64[ns, UTC]
name                                       object
eth_total                                 float64
usd_total                                 float64
buyer                                      object
tokenid                                     int64
project_type                               object
projectid                                   int64
platform                                   object
normalized_price                          float64
latterMintWindowInMins                    float64
lastMintTime                  datetime64[ns, UTC]
isWithin2hFromLastMintTime                   bool
dtype: object

,eth_total,usd_total,tokenid,projectid,normalized_price,latterMintWindowInMins,isWithin2hFromLastMintTime
count,5.283900e+04,5.012700e+04,5.283900e+04,52839.000000,5.283900e+04,52839.000000,52839
unique,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,False
freq,NaN,NaN,NaN,NaN,NaN,NaN,44522
mean,1.456965e+00,3.247039e+03,2.267791e+08,226.778497,5.479018e+00,32687.524914,NaN
std,5.877301e+00,1.367103e+04,9.136678e+07,91.367192,4.286629e+01,141818.945162,NaN
min,2.000000e-12,2.940910e-08,4.000000e+00,0.000000,1.186048e-11,0.250000,NaN
25%,1.390000e-01,2.688836e+02,1.670002e+08,167.000000,6.047589e-01,1.500000,NaN
50%,3.100000e-01,6.641118e+02,2.550003e+08,255.000000,1.167098e+00,2.850000,NaN
75%,1.000000e+00,2.284346e+03,2.890008e+08,289.000000,2.088414e+00,35.750000,NaN


,time,name,eth_total,usd_total,buyer,tokenid,project_type,projectid,platform,normalized_price,latterMintWindowInMins,lastMintTime,isWithin2hFromLastMintTime
52893,2022-01-01 00:00:00+00:00,Flowers by RVig,0.075,276.15900,\x4dcf0d851e8142,116000553,Factory,116,OpenSea,0.444768,44.15,2021-08-02 17:06:52+00:00,False
52892,2022-01-01 00:00:00+00:00,Organized Disruption,0.010,36.82120,\x791bf46d6aa113,133000016,Factory,133,OpenSea,0.035525,5.32,2021-08-11 20:34:28+00:00,False
52891,2022-01-01 00:02:00+00:00,Algobots by Stina Jo,1.750,6450.37750,\xa56c04347abee4,40000287,Curated,40,OpenSea,6.797341,2.85,2021-04-10 17:05:09+00:00,False
52890,2022-01-01 00:04:00+00:00,Skulptuur by Piter P,1.300,4791.70900,\xe84c5c241ed5e8,173000810,Curated,173,OpenSea,0.224433,2.42,2021-09-27 16:29:09+00:00,False
52889,2022-01-01 00:04:00+00:00,Andradite by Eltono,0.001,3.68593,\xd5caa4bd2ff510,71000152,Factory,71,OpenSea,0.009643,11358.65,2021-06-26 20:05:29+00:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-08-28 23:22:00+00:00,CatBlocks by Kristy,0.330,483.58200,\x0e63d7e4893630,73000236,Factory,73,OpenSea,2.978376,79.85,2021-05-29 18:48:49+00:00,False
3,2022-08-28 23:26:00+00:00,Balletic by Motus Ar,0.130,189.88060,\x52d77a8187160e,343000155,Factory,343,OpenSea,1.213782,1.53,2022-08-12 17:34:32+00:00,False
2,2022-08-28 23:34:00+00:00,Alan Ki Aankhen by F,0.880,1284.12240,\x29629f8b5e8b95,333000154,Curated,333,OpenSea,0.395376,0.28,2022-07-27 17:24:10+00:00,False
1,2022-08-28 23:35:00+00:00,Alan Ki Aankhen by F,1.188,1730.63088,\x29629f8b5e8b95,333000348,Curated,333,OpenSea,0.533758,0.28,2022-07-27 17:24:10+00:00,False


In [8]:
tradesByProjectId = p.groupby("projectid")
tradesByProjectId = pd.DataFrame({
    "tradeCount": tradesByProjectId["projectid"].count(),
    "tradeCount2hr": tradesByProjectId["isWithin2hFromLastMintTime"].sum(),
    "medianNormPrice2h": tradesByProjectId.apply(lambda df: df[df["isWithin2hFromLastMintTime"]].normalized_price.median()),
    "meanNormPrice2h": tradesByProjectId.apply(lambda df: df[df["isWithin2hFromLastMintTime"]].normalized_price.mean()),
    "projectType": tradesByProjectId["project_type"].first(),
})
tradesByProjectId = tradesByProjectId.reset_index()
tradesByProjectId["latterMintWindowInMins"] = tradesByProjectId["projectid"].map(mintsByProjectId.latterMintWindowInMins)

# removing 0 sales projects -> very old ones
tradesByProjectId = tradesByProjectId[tradesByProjectId["tradeCount2hr"] > 0]

display("number of collections", len(tradesByProjectId.index))
groupedTradesByProjectId = tradesByProjectId.groupby("projectType")
tradesStats = pd.DataFrame({
    "averageTradesWithin2hr": groupedTradesByProjectId["tradeCount2hr"].mean(),
    "minTradesWithin2hr": groupedTradesByProjectId["tradeCount2hr"].min(),
    "maxTradesWithin2hr": groupedTradesByProjectId["tradeCount2hr"].max(),
    "medianTradesWithin2hr": groupedTradesByProjectId["tradeCount2hr"].median(),
})
tradesStats

'number of collections'

87

,averageTradesWithin2hr,minTradesWithin2hr,maxTradesWithin2hr,medianTradesWithin2hr
projectType,,,,
Curated,195.111111,2,427,202.0
Factory,63.571429,1,542,26.0
Playground,95.769231,1,349,47.0


In [9]:
def update(projectid = 331):
    pId = p[p["projectid"] == projectid]
    display("project type: ", pId["project_type"].iloc[0])
    pId[pId["isWithin2hFromLastMintTime"]].plot(x="time", y=["normalized_price", "eth_total"], figsize=(20,8))

interact(update)

interactive(children=(IntSlider(value=331, description='projectid', max=993, min=-331), Output()), _dom_classe…

<function __main__.update(projectid=331)>

In [10]:
import matplotlib.pyplot as plt
medianMintWindow = tradesByProjectId["latterMintWindowInMins"].median()
firstQuantileMintWindow = tradesByProjectId["latterMintWindowInMins"].quantile(.25)
thirdQuantileMintWindow = tradesByProjectId["latterMintWindowInMins"].quantile(.75)
maxMintWindow = tradesByProjectId["latterMintWindowInMins"].max()
mintWindowValues = [firstQuantileMintWindow, medianMintWindow, thirdQuantileMintWindow, maxMintWindow]
collectionType = ["Curated", "Playground", "Factory"]
colors = ["green", "blue", "red"]
def updateWindow(i = 1):
    fig, axs = plt.subplots(2)
    displayedTrades = tradesByProjectId[tradesByProjectId["latterMintWindowInMins"] <= mintWindowValues[i]]

    # removing high profit collection to get a better view on more "casual" collections
    displayedTrades = displayedTrades[displayedTrades["medianNormPrice2h"] < 3]

    print("number of collections:", displayedTrades["projectType"].count())
    for idx,v in enumerate(collectionType):
        print(colors[idx])
        print(v)
        x = displayedTrades[displayedTrades["projectType"] == v].latterMintWindowInMins
        y = displayedTrades[displayedTrades["projectType"] == v].tradeCount2hr
        y2 = displayedTrades[displayedTrades["projectType"] == v].medianNormPrice2h
        axs[0].set_xlabel("mint window(minutes)")
        axs[0].set_ylabel("trade count within 2 hours after last mint")
        axs[1].axhline(y=1, color='r', linestyle='-')
        axs[1].set_xlabel("mint window(minutes)")
        axs[1].set_ylabel("median normalised price within 2 hours after last mint")
        axs[0].scatter(x, y, color=colors[idx])
        axs[1].scatter(x, y2, color=colors[idx])

interact(updateWindow)

interactive(children=(IntSlider(value=1, description='i', max=3, min=-1), Output()), _dom_classes=('widget-int…

<function __main__.updateWindow(i=1)>

In [13]:
def updateProfit(mintWindow = 120, withinMins = 120):
    #creating custom time selector in the main dataset
    p["isWithinxFromLastMintTime"] = ((p["time"] - p["lastMintTime"]).dt.total_seconds() / 60 < withinMins) & ((p["time"] - p["lastMintTime"]).dt.total_seconds() > 0)
    print(p["lastMintTime"])
    
    #adding it to my tradesByProjectId set
    pgrouped = p.groupby("projectid")
    pgrouped = pd.DataFrame({
        "withinXMins": pgrouped.apply(lambda df: df[df["isWithinxFromLastMintTime"]].normalized_price.median()),
    })
    tradesByProjectId["withinXMins"] = tradesByProjectId["projectid"].map(pgrouped.withinXMins)

    #selecting the matching latterMintWindow
    soldOut = tradesByProjectId[tradesByProjectId["latterMintWindowInMins"] <= mintWindow]

    #displaying data
    print("number of collections:", soldOut["projectid"].count())
    profits = soldOut[soldOut["withinXMins"] >= 1.1].projectid.count()
    loss = soldOut[soldOut["withinXMins"] < 1.1].projectid.count()
    print("for collections with a latterMintWindow <=", mintWindow, "mins")
    print("(based on median price within", withinMins, "mins after last mint time)")
    print("number of profits:", profits)
    print("number of losses:", loss)
    print("profit rate(%):", profits / (profits + loss) * 100)

interact(updateProfit)

interactive(children=(IntSlider(value=120, description='mintWindow', max=360, min=-120), IntSlider(value=120, …

<function __main__.updateProfit(mintWindow=120, withinMins=120)>

In [12]:
def updateProfitWindow(mintWindow = 120, startWindow = 60, endWindow = 70):
    #creating custom time selector in the main dataset
    p["isWithinWindowAfterMint"] = ((p["time"] - p["lastMintTime"]).dt.total_seconds() / 60 > startWindow) & ((p["time"] - p["lastMintTime"]).dt.total_seconds() / 60 < endWindow)
    
    #adding it to my tradesByProjectId set
    pgrouped = p.groupby("projectid")
    pgrouped = pd.DataFrame({
        "withinWindow": pgrouped.apply(lambda df: df[df["isWithinWindowAfterMint"]].normalized_price.median()),
    })
    tradesByProjectId["withinWindow"] = tradesByProjectId["projectid"].map(pgrouped.withinWindow)

    #selecting the matching latterMintWindow
    soldOut = tradesByProjectId[tradesByProjectId["latterMintWindowInMins"] <= mintWindow]

    #displaying data
    print("number of collections:", soldOut["projectid"].count())
    profits = soldOut[soldOut["withinWindow"] >= 1.1].projectid.count()
    loss = soldOut[soldOut["withinWindow"] < 1.1].projectid.count()
    print("for collections with a latterMintWindow <=", mintWindow, "mins")
    print("(based on median price within", startWindow, "mins after last mint time to", endWindow, "mins after mint)")
    print("number of profits:", profits)
    print("number of losses:", loss)
    print("profit rate(%):", profits / (profits + loss) * 100)

interact(updateProfitWindow)

interactive(children=(IntSlider(value=120, description='mintWindow', max=360, min=-120), IntSlider(value=60, d…

<function __main__.updateProfitWindow(mintWindow=120, startWindow=60, endWindow=70)>